# Data ETL
> ETL - Extract, Transform, Load: Gather raw data from various sources, clean and consolidate it, and then load it into a single centralized destination
> - Extract data from various sources (databases, files, APIs, etc.)
> - Transform the data by cleaning, formatting, or combining it as needed
> - Load the processed data into a destination, such as a database, data warehouse, or analytics tool
> ETL is a foundational step in data engineering and analytics pipelines, ensuring data is ready for analysis or further processing.

## Initialize

In [1]:
import pandas as pd
import numpy as np
# import polars as pl
import gdown
import os
import rich

In [2]:
# ## Onetime data downloading
# model_file_id = "15DamSAHtEUsLn2qwnwmJFgD7Djk1MizJ"
# data_folder = "c:/teaching/fall2025/data_science_bootcamp_lecture_1"
# gdown.download(
#     f"https://drive.google.com/uc?id={model_file_id}",
#     f'{data_folder}/.local/auto_policies_2017.csv', 
#     quiet=False)
# inference_file_id = "1ZppcSp8WMinV3iUdNUDapfDg3bMEqMat"
# gdown.download(
#     f"https://drive.google.com/uc?id={inference_file_id}",
#     f'{data_folder}/.local/auto_potential_customers_2018.csv', 
#     quiet=False)

## Extract Data

In [3]:
data_folder = "c:/teaching/fall2025/data_science_bootcamp_lecture_1"
model_data = pd.read_csv(f'{data_folder}/.local/auto_policies_2017.csv')
print( model_data.shape )
model_data.head(5)

(60392, 15)


,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium
0,64080188,7/10/2017,M,2.0,1/4/1989,631.0,B,140.9,4,TRUCK,0.924,NaN,0,0.000000,716.53
1,18917133,7/31/2017,M,2.0,6/21/1985,531.0,C,136.5,3,HBACK,1.430,B,1,583.010876,716.53
2,82742606,2/1/2017,M,6.0,7/25/1942,838.0,D,88.8,3,SEDAN,1.100,D,1,159.375757,716.53
3,43601997,10/17/2017,M,5.0,6/8/1959,835.0,E,NaN,2,SEDAN,2.090,NaN,0,0.000000,716.53
4,58746861,4/13/2017,F,4.0,5/16/1967,748.0,C,123.0,3,HBACK,0.803,C,1,143.555642,716.53


In [4]:
inference_data = pd.read_csv(f'{data_folder}/.local/auto_potential_customers_2018.csv')
print( inference_data.shape )
inference_data.head(5)

(7464, 10)


,quote_number,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value
0,29323463,F,4.0,4/14/1968,750.0,C,124.5,2,HBACK,1.100
1,16732441,M,2.0,10/4/1984,567.0,F,NaN,3,STNWG,2.079
2,68744228,M,4.0,11/30/1962,378.0,D,111.4,3,HDTOP,2.354
3,30875047,F,3.0,5/17/1973,807.0,E,NaN,4,UTE,1.969
4,37049210,M,6.0,10/2/1945,844.0,C,111.0,1,HBACK,1.969


## Transform Data
### Visualize data

In [5]:
output_folder = "../.local/analysis_pipeline/data_etl"
os.makedirs(output_folder, exist_ok=True)

In [6]:
from ydata_profiling import ProfileReport 

model_data_profile = ProfileReport(model_data, title="Model Data Profiling Report")

In [7]:
# model_data_profile.to_notebook_iframe()
model_data_profile.to_file(f"{output_folder}/model_data_profile.html") 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:00<00:00, 25.07it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Clean data

In [8]:
## Show column information for abnormal checking purpose
rich.print( model_data.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60392 entries, 0 to 60391
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pol_number      60392 non-null  int64  
 1   pol_eff_dt      60392 non-null  object 
 2   gender          60392 non-null  object 
 3   agecat          55561 non-null  float64
 4   date_of_birth   60392 non-null  object 
 5   credit_score    57591 non-null  float64
 6   area            60392 non-null  object 
 7   traffic_index   56889 non-null  float64
 8   veh_age         60392 non-null  int64  
 9   veh_body        60392 non-null  object 
 10  veh_value       60392 non-null  float64
 11  claim_office    10030 non-null  object 
 12  numclaims       60392 non-null  int64  
 13  claimcst0       60392 non-null  float64
 14  annual_premium  60392 non-null  float64
dtypes: float64(6), int64(3), object(6)
memory usage: 6.9+ MB


None

In [9]:
## Format data into correct types
# Convert 'date_of_birth' to datetime, coercing errors to NaT for invalid formats
model_data['pol_eff_dt'] = pd.to_datetime(model_data['pol_eff_dt'], format='%m/%d/%Y', errors='coerce')
model_data['date_of_birth'] = pd.to_datetime(model_data['date_of_birth'], format='%m/%d/%Y', errors='coerce')

In [10]:
## Validate 'date_of_birth'
# Show 'date_of_birth' for null 'agecat'
rich.print( model_data.loc[model_data['agecat'].isnull(), ['date_of_birth']].describe() )
# Find 'agecat' cutoff for each 'agecat'
# Here, we are giving an example on 'agecat' = 1
rich.print( model_data.loc[model_data['agecat'].isin([1]), ['date_of_birth']].describe() )

date_of_birth
count                           4831
mean   1970-01-19 17:50:05.340509211
min              1923-05-29 00:00:00
25%              1960-02-16 12:00:00
50%              1970-09-08 00:00:00
75%              1981-08-26 12:00:00
max              1999-12-14 00:00:00

date_of_birth
count                           4685
mean   1995-01-15 14:30:27.278548608
min              1990-01-01 00:00:00
25%              1992-07-09 00:00:00
50%              1995-01-12 00:00:00
75%              1997-07-22 00:00:00
max              1999-12-31 00:00:00

In [11]:
## Correct 'date_of_birth'
# Define bins and labels for age categories
bins = [
    pd.Timestamp('1900-01-01'), 
    pd.Timestamp('1950-01-01'), 
    pd.Timestamp('1960-01-01'),
    pd.Timestamp('1970-01-01'), 
    pd.Timestamp('1980-01-01'), 
    pd.Timestamp('1990-01-01'), 
    pd.Timestamp.max 
    ]
labels = [1, 2, 3, 4, 5, 6]
# Use pd.cut to assign agecat2 based on date_of_birth
model_data['agecat2'] = pd.cut(model_data['date_of_birth'], bins=bins, labels=labels, right=False)
# Ensure float type for consistency
model_data['agecat2'] = model_data['agecat2'].astype(float)  

### Consolidate Data

In [12]:
# Check category distribution of 'veh_body'
rich.print( model_data['veh_body'].value_counts() )
# Group 'MCARA', 'CONVT', 'BUS', and 'RDSTR' 'veh_body' as 'Other'
model_data.loc[model_data['veh_body'].isin(['MCARA','CONVT','BUS','RDSTR']), 'veh_body'] = 'Other'

veh_body
SEDAN    19798
HBACK    16793
STNWG    14487
UTE       4101
TRUCK     1561
HDTOP     1403
COUPE      698
PANVN      663
MIBUS      639
MCARA      111
CONVT       74
BUS         42
RDSTR       22
Name: count, dtype: int64

In [13]:
# Add a year and quarter variable for later consistency check purpose
model_data['pol_year'] = model_data['pol_eff_dt'].dt.year
model_data['pol_quarter'] = model_data['pol_eff_dt'].dt.quarter

In [14]:
# Cap 'veh_value' and 'traffic_index' at the 99th percentile for outlier control
veh_value_cap = round(np.nanpercentile(model_data['veh_value'], 99), 3)
traffic_index_cap = round(np.nanpercentile(model_data['traffic_index'], 99), 3)
print(f"veh_value cap at 99th percentile: {veh_value_cap}")
print(f"traffic_index cap at 99th percentile: {traffic_index_cap}")
model_data['veh_value'] = model_data['veh_value'].clip(upper=veh_value_cap)
model_data['traffic_index'] = model_data['traffic_index'].clip(upper=traffic_index_cap)

veh_value cap at 99th percentile: 6.315
traffic_index cap at 99th percentile: 160.0


In [15]:
# Assume single vehicle policy and create a vehicle count variable
model_data['veh_cnt'] = 1

# Add policy year 
model_data['pol_year'] = 2017

### Carry the above steps to inference data 

In [ ]:
import sys
sys.path.append(os.path.abspath(".."))
from analysis_tool_chest.data_etl import DataETL

output_folder = "../.local/analysis_pipeline/data_etl"
os.makedirs(output_folder, exist_ok=True)
etl = DataETL(inference_data)
etl.profile_analysis(output_folder, file_name="inference_data_profile.html", title="Inference Data Profiling Report")
processed_inference_data = etl.transform(cap_dict={'veh_value': veh_value_cap, 'traffic_index': traffic_index_cap})

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 149.25it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

veh_value capped at 6.315
traffic_index capped at 160.0


## Load Data Into Memory

In [18]:
output_folder = "../.local/analysis_pipeline/data_etl"
model_data.to_parquet(f"{output_folder}/model_data.parquet", index=False)
processed_inference_data.to_parquet(f"{output_folder}/inference_data.parquet", index=False)